# Week 1 Day 2 - Webpage Summarizer using Ollama (llama3.2)

This notebook upgrades the Day 1 project to use an open-source local model via Ollama instead of OpenAI.

- Model: `llama3.2` (or `llama3.2:1b` if your machine is slower)
- Endpoint: `http://localhost:11434/v1` (OpenAI-compatible)
- No API charges; data stays on your machine



In [1]:
# Setup Ollama OpenAI-compatible client
import os
from openai import OpenAI
from dotenv import load_dotenv

# Optional .env for other configs
load_dotenv(override=True)

OLLAMA_BASE_URL = "http://localhost:11434/v1"
MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")  

# Create client pointing to Ollama endpoint (api_key can be any non-empty string)
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

print(f"Client ready. Using model: {MODEL}")


Client ready. Using model: llama3.2


In [2]:
# Minimal scraper utilities (requests + BeautifulSoup)
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    )
}

def fetch_website_contents(url, char_limit=2000):
    try:
        r = requests.get(url, headers=HEADERS, timeout=15)
        r.raise_for_status()
        html = r.text
    except Exception as e:
        return f"Error fetching {url}: {e}"

    soup = BeautifulSoup(html, "html.parser")

    # Remove scripts/styles
    for el in soup(["script", "style", "noscript", "template"]):
        el.decompose()

    title = soup.title.get_text(strip=True) if soup.title else "No title"
    text = soup.get_text("\n")
    # Basic whitespace cleanup
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    text = "\n".join(lines)

    return (f"{title}\n\n{text}")[:char_limit]

print("Scraper ready.")


Scraper ready.


In [3]:
# Summarization with llama3.2 via Ollama's OpenAI-compatible API
from IPython.display import Markdown, display

def summarize_url(url, model=MODEL, temperature=0.4, max_tokens=400):
    website = fetch_website_contents(url, char_limit=3000)

    system_prompt = (
        "You are a helpful assistant that analyzes a website's textual content "
        "and produces a clear, concise markdown summary with bullet points."
    )

    user_prompt = f"""
Here are the contents of a website.
Provide a short summary of this website in markdown.
Include key sections, offerings, and any notable announcements.

{website}
"""

    resp = ollama.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return resp.choices[0].message.content


def display_summary(url, **kwargs):
    print(f"Summarizing with {MODEL} @ Ollama → {url}")
    md = summarize_url(url, **kwargs)
    display(Markdown(md))

print("Summarizer ready.")


Summarizer ready.


In [4]:
# Try it out
TEST_URL = "https://the-star.co.ke"  # change to any site you want

display_summary(TEST_URL, temperature=0.4, max_tokens=400)


Summarizing with llama3.2 @ Ollama → https://the-star.co.ke


**The Star Website Summary**
=====================================

### Key Sections

*   **News**: Latest news articles, including updates on Mashujaa Day celebrations, politics, business, health, and more.
*   **Podcasts**: Audio content available for listening or download.
*   **In-pictures**: Galleries of photos related to various topics.

### Offerings

*   **Mashujaa Day Coverage**: In-depth coverage of the Kenya's Mashujaa Day celebrations, including news articles, performances, and events.
*   **Raila Odinga Tribute**: Articles and features honoring the late former President Raila Odinga, including his life, legacy, and impact on Kenyan politics.

### Notable Announcements

*   **KQ-Qatar Airways Deal**: Partnership to open travel to 19 destinations, effective October 26, 2025.
*   **Raila's State Burial**: Details of the state funeral ceremony held for former President Raila Odinga.
*   **Mashujaa Day Performances**: Special performances honoring Raila Odinga, including music and cultural events.

### Other Key Content

*   **Editorials and Op-Eds**: Articles on current events, politics, and social issues from various perspectives.
*   **Infographics**: Visual representations of data and information on topics such as Kenyan leaders who have been accorded state funeral, phrases Raila Odinga loved, and more.